In [25]:
import geopandas as gpd
import random
import fiona
from shapely.geometry import Point

In [14]:
gpkg = './lab3.gpkg'
layers = fiona.listlayers(gpkg)

In [15]:
random.seed(0)

In [16]:
soils = [layer for layer in layers if 'ssurgo' in layer]
sheds = [layer for layer in layers if 'wdb' in layer]

In [70]:
ssurgo_gdf = gpd.read_file(gpkg, layer=soils[0])
for shed in sheds:
    point_samples = {'huc': [], 'geometry': []}
    
    shed_gdf = gpd.read_file(gpkg, layer=shed)
    huc_column_name = [column for column in shed_gdf.columns if 'HUC' in column][0]
    for idx, row in shed_gdf.iterrows():
        # https://geopandas.org/reference.html#geopandas.GeoSeries.bounds
        extent = row.geometry.bounds
        density = round(0.05 * row.geometry.area / 1000000)
        cntr = 0
        while cntr < density:
            point = Point(random.uniform(extent[0], extent[2]), random.uniform(extent[1], extent[3]))
            if point.within(row.geometry):
                point_samples['huc'].append(row[huc_column_name][:8])
                point_samples['geometry'].append(point)
                cntr += 1

    point_samples_gdf = gpd.GeoDataFrame(point_samples, crs=shed_gdf.crs)
    
    inter = gpd.overlay(point_samples_gdf, ssurgo_gdf, how='intersection')
    
    summary = inter.groupby(by='huc').mean()
    for idx, row in summary.iterrows():
        print(shed, 'sampling of aws0150 for watershed: ', idx, 'was: ', row['aws0150'])

wdbhuc8 sampling of aws0150 for watershed:  10190005 was:  11.298181818181826
wdbhuc8 sampling of aws0150 for watershed:  10190006 was:  10.99811881188119
wdbhuc8 sampling of aws0150 for watershed:  10190007 was:  10.937999999999997
wdbhuc12 sampling of aws0150 for watershed:  10190005 was:  11.330305343511455
wdbhuc12 sampling of aws0150 for watershed:  10190006 was:  10.746725663716816
wdbhuc12 sampling of aws0150 for watershed:  10190007 was:  11.475400843881843


In [61]:
inter = gpd.overlay(point_samples_gdf, ssurgo_gdf, how='intersection')

wdbhuc12 sampling of aws0150 for watershed:  10190005 was:  11.024422310756975
wdbhuc12 sampling of aws0150 for watershed:  10190006 was:  10.837808219178083
wdbhuc12 sampling of aws0150 for watershed:  10190007 was:  10.725648535564854


In [68]:
summary

,aws0150,Shape_Length,Shape_Area
huc,,,
10190005,11.024422,23524.580339,4.164995e+06
10190006,10.837808,17515.119339,3.064778e+06
10190007,10.725649,36316.490198,6.082243e+06
